# Proyecto ML Taxi. NB1: Limpieza de datos y creación de dataframe

### En este notebook se describe el proceso de limpieza y generación del dataframe final con el que se trabajarán los modelos de ML.

### Obtención del dataset
Se carga dataframe con los datos recuperados de (https://huggingface.co/datasets/Chendi/NYC_TAXI_FARE_CLEANED)

In [1]:
# Importa la función 'load_dataset' de la librería 'datasets', que permite cargar datasets de Hugging Face.
from datasets import load_dataset  
# Carga el dataset "Chendi/NYC_TAXI_FARE_CLEANED" desde Hugging Face en su formato predeterminado (diccionario de datasets)
df = load_dataset("Chendi/NYC_TAXI_FARE_CLEANED")  
# Convierte el dataset de Hugging Face (que está en formato 'DatasetDict') a un DataFrame de pandas.
df = df["train"].to_pandas()  

c:\Users\Gerardo\anaconda33\envs\en_taxi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Limpieza de datos 

In [6]:
#Se eliminan valores nulos
from functions_limpieza import limpieza1
df=limpieza1(df)

Se eliminaron 0 por valores nulos. Restan 54315620 datos.


### Reducción del dataframe 
Dada la cantidad de datos original, se decide trabajar con una muestra menor.

In [5]:
df_sample=df.sample(100000) #Se toma una muestra de 100k de datos del data originial.
df_sample.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
13663773,7.3,2010-10-24 12:49:03 UTC,-74.008046,40.723442,-74.017175,40.705310,4
18227425,12.0,2013-09-03 10:07:00 UTC,-74.011285,40.723892,-73.994230,40.743457,1
2755683,10.5,2014-04-02 13:30:00 UTC,-73.991072,40.744547,-73.984437,40.757920,1
48326621,39.8,2011-09-25 13:38:04 UTC,-73.986798,40.761388,-73.870965,40.771974,1
42771250,3.5,2013-01-18 04:51:00 UTC,-73.916373,40.762205,-73.920967,40.756542,1


### Feature engineering
A continuación se crean algunas varibles nuevas, provenientes de los datos base. Importante tener en cuenta que no se generan variables relacionadas con la tarifa del viaje, dado que esa es la característica a predecir.

In [7]:
from functions_limpieza import ft_eng1
df = ft_eng1(df_sample) #se aplica función para añadir columnas con variables calculadas a partir de las variables iniciales


 Se eliminó la columna 'pickup_datetime'. Las columnas en el dataframe son: Index(['fare_amount', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
       'pickup_hour', 'pickup_minutes', 'peak_hour', 'pickup_year',
       'pickup_month', 'pickup_day', 'day_of_week', 'distance'],
      dtype='object').


In [8]:
#Visualización del dataframe generado
df.sample(5)

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_hour,pickup_minutes,peak_hour,pickup_year,pickup_month,pickup_day,day_of_week,distance
38215056,3.7,-73.985900,40.749002,-73.985900,40.749002,1,22,30,False,2009,1,5,0,0.000000
27800720,7.5,-73.975910,40.751161,-73.981978,40.761217,1,18,18,True,2009,3,19,3,1.229450
47863309,7.3,-73.965828,40.754077,-73.954424,40.766071,1,10,20,False,2010,1,21,3,1.643544
26438755,4.5,-73.987277,40.750633,-73.988780,40.757847,2,15,43,False,2010,12,14,1,0.812089
49692413,5.3,-73.967541,40.769070,-73.955260,40.777133,1,17,15,True,2009,1,24,5,1.368691


Las variables creadas involucran datos cícilicos temporales como horas, minutos, meses y días. Por esta razón y con mira en el análisis de machine learning, se aplicará una transformación trigonométrica. Como resultado, cada variable será representada en un espacio bidimensional por medio de valores de senos y cosenos.

In [9]:
#se define lista con las columnas que representan variables cíclicas
variables_ciclicas=['pickup_hour', 'pickup_minutes','pickup_month', 'pickup_day', 'day_of_week']

from functions_limpieza import ft_eng2
df_final=ft_eng2(df,variables_ciclicas) # Se aplica función para transformación de variables cíclicas

In [9]:
#Visualización del dataframe generado
df_final.sample(5)

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,peak_hour,pickup_year,distance,pickup_hour_sin,pickup_hour_cos,pickup_minutes_sin,pickup_minutes_cos,pickup_month_sin,pickup_month_cos,pickup_day_sin,pickup_day_cos,day_of_week_sin,day_of_week_cos
53475545,6.9,-74.001419,40.751460,-74.001444,40.751462,1,False,2010,0.002118,5.195840e-01,0.854419,0.638244,-0.769834,5.000000e-01,-0.866025,0.201299,0.979530,-8.660254e-01,0.5
40783887,23.3,-73.987240,40.733139,-73.868078,40.722865,1,False,2010,10.105993,0.000000e+00,1.000000,-0.364161,-0.931336,8.660254e-01,-0.500000,0.897805,-0.440394,-8.660254e-01,0.5
53933794,24.9,-73.869077,40.771335,-73.950917,40.777597,1,True,2011,6.926537,-9.976688e-01,-0.068242,0.638244,-0.769834,-5.000000e-01,-0.866025,-0.299363,-0.954139,-2.449294e-16,1.0
30586896,18.5,-73.957545,40.765718,-74.001097,40.746572,1,False,2009,4.241379,-2.449294e-16,1.000000,0.053222,-0.998583,-2.449294e-16,1.000000,0.651372,-0.758758,-8.660254e-01,0.5
7456352,2.5,-73.982010,40.758026,-73.982018,40.758034,1,False,2015,0.001064,-7.308360e-01,0.682553,-0.314077,0.949398,5.000000e-01,0.866025,0.651372,-0.758758,0.000000e+00,1.0


### Creación de datafreme final en archivo csv

In [10]:
#Conversión del dataframe final a archivo csv
df_final.to_csv('data/df_final.csv', index=False, encoding='utf-8')